# Start - download

In [29]:
# !pip install pyspark

In [30]:
# !pip install -U -q PyDrive

In [31]:
# sudo apt install openjdk-11-jdk-headless -qq 
# đã cài đặt jdk mặc định từ repository

# Activity 1.


In [32]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
# from pyspark.sql import functions

In [33]:
# Tạo hàm loadMovieNames() để đọc file:
def loadMovieNames():
  movieNames = {}
  with open("ml-100k/u.item", encoding="latin1") as f:
    for line in f:
      fields = line.split('|')
      movieNames[int(fields[0])] = fields[1]
  return movieNames

movieNames = loadMovieNames()
# print(movieNames)
# Print 5 element of dictionary 
for idx, k in enumerate(movieNames):
  if idx == 5: break
  print((k, movieNames[k]))

(1, 'Toy Story (1995)')
(2, 'GoldenEye (1995)')
(3, 'Four Rooms (1995)')
(4, 'Get Shorty (1995)')
(5, 'Copycat (1995)')


In [34]:
# Khởi tạo hàm parseInput():
def parseInput(line):
  fields = line.split()
  return Row(movieID = int(fields[1]), rating = float(fields[2]))

In [35]:
# Sử dụng SparkSession để tạo DataFrame dạng bảng. Thực thi SQL trên bảng.
spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

In [36]:
# Đọc file
lines = spark.sparkContext.textFile("ml-100k/u.data")

In [37]:
# Sử dụng hàm parseInput() đã khởi tạo ở trên và mapping với dữ liệu.
movies = lines.map(parseInput)
movies.take(5)

[Row(movieID=242, rating=3.0),
 Row(movieID=302, rating=3.0),
 Row(movieID=377, rating=1.0),
 Row(movieID=51, rating=2.0),
 Row(movieID=346, rating=1.0)]

In [38]:
# Khởi tạo dataFrame có tên movieDataset:
movieDataset = spark.createDataFrame(movies)
movieDataset.take(5)

[Row(movieID=242, rating=3.0),
 Row(movieID=302, rating=3.0),
 Row(movieID=377, rating=1.0),
 Row(movieID=51, rating=2.0),
 Row(movieID=346, rating=1.0)]

In [39]:
# Tính trung bình điểm rating của các movieID từ movieDataset:
averageRatings = movieDataset.groupBy("movieID").avg("rating")
averageRatings.take(5)

[Row(movieID=474, avg(rating)=4.252577319587629),
 Row(movieID=29, avg(rating)=2.6666666666666665),
 Row(movieID=26, avg(rating)=3.452054794520548),
 Row(movieID=964, avg(rating)=3.3333333333333335),
 Row(movieID=65, avg(rating)=3.5391304347826087)]

In [40]:
# Đếm số lượt rate cho từng movieID:
counts = movieDataset.groupBy("movieID").count()
counts.take(5)

[Row(movieID=474, count=194),
 Row(movieID=29, count=114),
 Row(movieID=26, count=73),
 Row(movieID=964, count=9),
 Row(movieID=65, count=115)]

In [41]:
# Kết hợp 2 bảng counts và averageRatings:
averagesAndCounts = counts.join(averageRatings, "movieID")
averagesAndCounts.take(5)

[Row(movieID=474, count=194, avg(rating)=4.252577319587629),
 Row(movieID=29, count=114, avg(rating)=2.6666666666666665),
 Row(movieID=26, count=73, avg(rating)=3.452054794520548),
 Row(movieID=964, count=9, avg(rating)=3.3333333333333335),
 Row(movieID=65, count=115, avg(rating)=3.5391304347826087)]

In [42]:
# Sắp xếp và hiển thị bảng averagesAndCounts theo avg(rating):
sorted_DF = averagesAndCounts.orderBy("avg(rating)")
topFive = sorted_DF.head(5)

for movie in topFive:
  print(movieNames[movie[0]], movie[1], movie[2])

# Stop the session
spark.stop()

Hostile Intentions (1994) 1 1.0
Bloody Child, The (1996) 1 1.0
Amityville: A New Generation (1993) 5 1.0
Lotto Land (1995) 1 1.0
Touki Bouki (Journey of the Hyena) (1973) 1 1.0


# Activity 2.


In [43]:
# Khai báo thư viện
from pyspark.sql import SparkSession

Sử dụng SparkContext tạo RDD từ file “u.data" với 2 trường thông tin “movieID” và “rating”. Khởi tạo hàm parseInput() để thực thi.

In [44]:
# Khởi tạo hàm parseInput():
def parseInput(line):
  fields = line.split()
  return (int(fields[1]), (float(fields[2]), 1.0)) # (MovieID, (Ratings, 1.0 - CountRate))

In [45]:
# Sử dụng SparkSession để tạo DataFrame dạng bảng. Thực thi SQL trên bảng.
spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

In [46]:
# Đọc file
lines = spark.sparkContext.textFile("ml-100k/u.data")
type(lines)

pyspark.rdd.RDD

In [47]:
# Sử dụng hàm parseInput() đã khởi tạo ở trên và mapping với dữ liệu.
movieRatings = lines.map(parseInput)
movieRatings.take(5)

[(242, (3.0, 1.0)),
 (302, (3.0, 1.0)),
 (377, (1.0, 1.0)),
 (51, (2.0, 1.0)),
 (346, (1.0, 1.0))]

Tạo RDD in ra chứa thông tin của từng movieID: (countRating, totalRating)

In [48]:
ratingTotalsAndCount = movieRatings.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
# x is Ratings, y = CountRate
ratingTotalsAndCount.take(5)

[(242, (467.0, 117.0)),
 (302, (1236.0, 297.0)),
 (346, (459.0, 126.0)),
 (474, (825.0, 194.0)),
 (86, (591.0, 150.0))]

Tạo RDD tính trung bình rating của từng movieID
movieID: (averageRatings)

In [49]:
# Tính trung bình điểm rating của các movieID từ ratingTotalsAndCount:
averageRatings = ratingTotalsAndCount.mapValues(lambda x: x[0] / x[1])
averageRatings.take(5)

[(242, 3.9914529914529915),
 (302, 4.161616161616162),
 (346, 3.642857142857143),
 (474, 4.252577319587629),
 (86, 3.94)]

Tạo RDD sắp xếp movieID theo averageRatings

In [50]:
# Sắp xếp và hiển thị bảng averagesAndCounts theo avg(rating):
sorted_RDD = averageRatings.sortBy(lambda x: (x[1],))
topFive = sorted_RDD.take(5)

print(topFive)

# Stop the session
spark.stop()

[(858, 1.0), (1334, 1.0), (1348, 1.0), (1320, 1.0), (314, 1.0)]


# Activity 3.

## Vấn đề 1.

In [51]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import Row

### B1. Nạp data

In [52]:
def parseInput(line):
  fields = line.split()
  return Row(userID = int(fields[0]), movieID = int(fields[1]), rate = int(fields[2]))

spark = SparkSession.builder.appName("RecommendMovie").getOrCreate()

# Đọc file
lines = spark.sparkContext.textFile("ml-100k/u.data")

# Sử dụng hàm parseInput() đã khởi tạo ở trên và mapping với dữ liệu.
users_movies = lines.map(parseInput)
users_movies_DSet = spark.createDataFrame(users_movies)

### B2. Tìm hiểu về dữ liệu

In [53]:
# Tìm kích thước để tạo ma trận 
users = users_movies_DSet.select('userID').distinct()
movies = users_movies_DSet.select('movieID').distinct()
user_number = users.count()
movie_number = movies.count()
print(user_number, movie_number)

943 1682


In [54]:
minUserID = users.agg({'userID': 'min'}).collect()[0][0]
maxUserID = users.agg({'userID': 'max'}).collect()[0][0]
minMovieID = movies.agg({'movieID': 'min'}).collect()[0][0]
maxMovieID = movies.agg({'movieID': 'max'}).collect()[0][0]
print(minUserID, maxUserID, minMovieID, maxMovieID)

1 943 1 1682


### B3. Tạo ma trận kết quả theo yêu cầu

In [55]:
# Sử dụng array của Numpy vì Matrix của Spark không cho phép sửa đổi Matrix sau khi tạo
X = np.zeros((user_number, movie_number + 1))
for index in range(user_number):
  X[index][0] = index + 1
for x in users_movies_DSet.collect():
  X[x.userID - 1][x.movieID] = x.rate

In [56]:
# Kiểm tra thử kết quả
print(X[:10, :10])

[[ 1.  5.  3.  4.  3.  3.  5.  4.  1.  5.]
 [ 2.  4.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 3.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 4.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 5.  4.  3.  0.  0.  0.  0.  0.  0.  0.]
 [ 6.  4.  0.  0.  0.  0.  0.  2.  4.  4.]
 [ 7.  0.  0.  0.  5.  0.  0.  5.  5.  5.]
 [ 8.  0.  0.  0.  0.  0.  0.  3.  0.  0.]
 [ 9.  0.  0.  0.  0.  0.  5.  4.  0.  0.]
 [10.  4.  0.  0.  4.  0.  0.  4.  0.  4.]]


## Vấn đề 2.

### B1. Dựng hàm tính khoảng cách các vector

In [57]:
def distance(x, y):
  from math import sqrt
  distance = 0
  for i in range(1, len(x)):
    distance += (x[i] - y[i]) ** 2
  return sqrt(distance)

def distance_other_user(userID, user_number, X):
  dictionary_suggestion = {}
  index = userID - 1
  for i in range(0, user_number):
    dictionary_suggestion[i + 1] = distance(X[index], X[i])
  return dictionary_suggestion

### B2. Dựng ma trận thể hiện khoảng cách với các người dùng

In [58]:
# Tính khoảng cách của user1 với các user còn lại
dictionary_suggestion = distance_other_user(1, user_number, X)
print(dictionary_suggestion)

{1: 0.0, 2: 65.25335240430181, 3: 65.91661399070799, 4: 65.36053855347276, 5: 61.14736298484179, 6: 63.34824385884742, 7: 78.689262291624, 8: 60.69596362197407, 9: 64.77653896280658, 10: 67.52777206453653, 11: 66.4304147209695, 12: 61.05735008989499, 13: 81.17881497040962, 14: 63.0317380372777, 15: 65.4446330878247, 16: 63.7808748764079, 17: 61.68468205316454, 18: 66.30987860040162, 19: 63.39558344238185, 20: 61.68468205316454, 21: 67.60177512462228, 22: 60.868711174132805, 23: 59.253691868102194, 24: 63.150613615387776, 25: 61.28621378417825, 26: 63.047601064592456, 27: 63.12685640834652, 28: 61.96773353931867, 29: 64.03124237432849, 30: 63.592452382338585, 31: 64.90762667052309, 32: 62.289646009589745, 33: 65.10760324263211, 34: 65.36818798161687, 35: 64.4437739428721, 36: 65.09992319503918, 37: 61.54673021371647, 38: 69.7137002317335, 39: 64.6297145282261, 40: 64.11708040764177, 41: 60.44832503882965, 42: 65.25335240430181, 43: 66.01514977639603, 44: 57.40209055426466, 45: 62.904689

## Vấn đề 3

In [63]:
# Đưa ra danh sách các phim đã được xem bởi user1
def loadMovieNames():
  movieNames = {}
  with open("ml-100k/u.item", encoding="latin1") as f:
    for line in f:
      fields = line.split('|')
      movieNames[int(fields[0])] = fields[1]
  return movieNames

movieNames = loadMovieNames()

def watched_movie_id(userID, X):
  result = []
  data = X[userID - 1]
  for i in range(1, len(data)):
     if data[i] != 0:
        result.append(i)
  return result

def watched_movie_name(userID, X, movieNames):
    result = []
    movieIDs = watched_movie_id(userID, X)
    for movieID in movieIDs:
       result.append(movieNames[movieID])
    return result 

In [64]:
print(watched_movie_name(1, X, movieNames)[:5])

['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', 'Get Shorty (1995)', 'Copycat (1995)']


## Vấn đề 4

In [65]:
def get_10_smallest_ids(dictionary):
  """
  Lấy ra 10 userID có giống người được khảo sát nhất.
  """
  # Sắp xếp dictionary theo giá trị của number theo thứ tự tăng dần.
  sorted_dictionary = sorted(dictionary.items(), key=lambda x: x[1])

  # Lấy ra 10 phần tử đầu tiên của dictionary sau khi sắp xếp.
  return [x[0] for x in sorted_dictionary[:10]]

def recommend_movie(userID, user_number, X, movieNames):
  dictionary_distance = distance_other_user(userID, user_number, X)
  list_similar_user = get_10_smallest_ids(dictionary_distance)  # chọn được 10 người dùng giống nhất

  dict_movie_similar = {}
  for user in list_similar_user:
    watched_movies = watched_movie_id(user, X)
    for movie in watched_movies:
      if movie in dict_movie_similar:
        dict_movie_similar[movie] += 1
      else:
        dict_movie_similar[movie] = 1
  # Lấy ra 10 phim được xem bởi 10 người giống nhất
  sorted_dict = sorted(dict_movie_similar.items(), key=lambda x: x[1], reverse=True)
  recommend_movie = [x[0] for x in sorted_dict[:10]]    
  for i in range(len(recommend_movie)):
    recommend_movie[i] = movieNames[recommend_movie[i]]
  return recommend_movie

In [67]:
print(recommend_movie(1, user_number, X, movieNames))

['Star Wars (1977)', 'Pulp Fiction (1994)', 'Blade Runner (1982)', 'Monty Python and the Holy Grail (1974)', 'Empire Strikes Back, The (1980)', 'Raiders of the Lost Ark (1981)', 'Aliens (1986)', 'Return of the Jedi (1983)', 'Alien (1979)', 'Back to the Future (1985)']
